In [1]:
from pyscf import gto, scf
import numpy as np    

In [2]:
# Define molecule: water molecule as an example
mol = gto.Mole()
mol.atom = '''
# O  0.0  0.0  0.0
H  0.0  0.8  0.6
H  0.0  0.8 -0.6
'''
mol.basis = 'sto-3g'  # Minimal basis set
mol.build()

# Run Hartree-Fock calculation
mf = scf.RHF(mol)
mf.kernel()

# Get the 1-electron and 2-electron reduced density matrices (RDMs)p
# 1-RDM in MO basis
rdm1 = mf.make_rdm1()

# 2-RDM in MO basis
rdm2 = mf.make_rdm2()

print(f"Hartree-Fock energy: {mf.e_tot:.8f} Hartree")
print(f"Converged: {mf.converged}")


converged SCF energy = -1.00510670656849
Hartree-Fock energy: -1.00510671 Hartree
Converged: True


In [3]:
# Get the two-electron integrals in MO basis
# Transform atomic orbital integrals to molecular orbital basis
mo_coeff = mf.mo_coeff
# Get 1-electron integrals in MO basis
h1_ao = mol.intor('int1e_kin') + mol.intor('int1e_nuc')
h1_mo = np.einsum('pq,pi,qj->ij', h1_ao, mo_coeff, mo_coeff)

# Get  Two-electron integrals in AO basis
eri_ao = mol.intor('int2e')  
eri_mo = np.einsum('pqrs,pi,qj,rk,sl->ijkl', eri_ao, 
                  mo_coeff, mo_coeff, mo_coeff, mo_coeff)



In [4]:
# Get nuclear energy
nuc_energy = mol.energy_nuc()

# Calculate 1-electron energy
one_elec_energy = np.trace(h1_ao@rdm1)

two_elec_energy_A = mf.e_tot - nuc_energy - one_elec_energy

print(one_elec_energy)

-2.0391701470806454


In [5]:

two_elec_energy_B = 0.5 * np.einsum('ijkl,ijkl', eri_ao, rdm2)
two_elec_energy_C = 0.5* ( np.trace(rdm1@mf.get_j()) - 0.5 * np.trace(rdm1@mf.get_k()))

In [6]:
print(two_elec_energy_A, mf.energy_elec()[1], two_elec_energy_B, two_elec_energy_C)

0.5930824314121552 0.5930824314121553 0.5930824314121553 0.5930824314121554


In [7]:
print(dir(mf))

['CASCI', 'CASSCF', 'CCSD', 'CISD', 'COSX', 'DDCOSMO', 'DFMP2', 'DIIS', 'Gradients', 'Hessian', 'MP2', 'QCISD', 'QMMM', 'TDA', 'TDHF', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_chkfile', '_eigh', '_eri', '_finalize', '_is_mem_enough', '_keys', '_opt', '_t0', '_transfer_attrs_', '_w0', 'analyze', 'apply', 'as_scanner', 'build', 'callback', 'canonicalize', 'check_convergence', 'check_sanity', 'chkfile', 'conv_check', 'conv_tol', 'conv_tol_cpscf', 'conv_tol_grad', 'converged', 'convert_from_', 'copy', 'damp', 'ddCOSMO', 'density_fit', 'diis', 'diis_damp', 'diis_file', 'diis_space', 'diis_space_rollback', 'diis_start_cycle', 'dip_moment', 'direct_sc